# Reconnaissance de caractères et de symboles

In [23]:
import numpy as np
import cv2
import os
from pyzbar import pyzbar

## Fonctions de détection dans une image

In [24]:
def decode_image(img):
    codes = pyzbar.decode(img)
    return codes

In [25]:
def update_image(img, code):
    (x, y, w, h) = code.rect
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)

    codeData = code.data.decode("utf-8")
    codeType = code.type

    text = "{} ({})".format(codeData, codeType)
    cv2.putText(img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    return img

In [42]:
def decode_and_update_image(original):
    updated = original.copy()
    img = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)

    codes = decode_image(img)
    for code in codes:
        updated = update_image(updated, code)

    return codes, original, updated

In [43]:
def decode_and_update_image_from_path(img_path):
    original = cv2.imread(img_path)

    return decode_and_update_image(original)

## Tests des fonctions

In [44]:
EMPTY_IMAGES_PATH = "./data/images/empty"
BARCODE_IMAGES_PATH = "./data/images/barcode"
QR_IMAGES_PATH = "./data/images/qrcode"

OUTPUT_PATH = "./data/outputs/"

In [45]:
def test_code_detection():
    # boucle sur les dossiers d'images
    for path in [EMPTY_IMAGES_PATH, BARCODE_IMAGES_PATH, QR_IMAGES_PATH]:
        # boucle sur les images du dossier
        for img_path in os.listdir(path):
            full_path = path + "/" + img_path
            print(full_path)

            codes, original, updated = decode_and_update_image_from_path(full_path)

            # print les codes trouvés
            for code in codes:
                codeData = code.data.decode("utf-8")
                codeType = code.type
                print("[INFO] Trouvé : {} code : {}".format(codeType, codeData))

            cv2.imwrite(OUTPUT_PATH + "/" + img_path, updated)

            print("\n")

In [46]:
test_code_detection()

./data/images/empty/img_001.jpg


./data/images/barcode/img_001.png
[INFO] Trouvé : EAN13 code : 0073935942400


./data/images/qrcode/Nestle-and-Google-launch-KitKat-QR-code.jpg
[INFO] Trouvé : QRCODE code : https://goo.gl/9aYrMF




## Fonctions de détection dans une vidéo

In [59]:
video_path = "./data/videos/v1.mp4"
output_path = "./data/outputs/output.mp4"

cap = cv2.VideoCapture(video_path)
width = int(cap.get(3))
height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height), True)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Erreur dans la lecture de la frame")
        break

    codes, original, updated = decode_and_update_image(frame)

    out.write(updated)

cap.release()
out.release()

OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Erreur dans la lecture de la frame
